<a href="https://colab.research.google.com/github/FifiSALIOU/blank-app/blob/main/SERIGNE_FALLOU_MBACKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install mistralai
%pip install faiss-gpu
%pip install faiss-cpu
%pip install PyPDF2
import os
from mistralai import Mistral
import numpy as np
import faiss
from getpass import getpass
import pickle
import json

ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [3]:
api_key = getpass("Entrez votre clé API Mistral:")
client = Mistral(api_key=api_key)

Entrez votre clé API Mistral:··········


In [28]:
# =============================================================================
# Fonction pour charger un fichier texte ou PDF
def load_file(file_path):
    _, ext = os.path.splitext(file_path)
    if ext.lower() == '.txt':
        with open(file_path, 'r', encoding='utf-8') as f:
            return f.read()
    elif ext.lower() == '.pdf':
        import PyPDF2
        text = ""
        with open(file_path, 'rb') as f:
            reader = PyPDF2.PdfReader(f)
            for page_num in range(len(reader.pages)):
                text += reader.pages[page_num].extract_text()
        return text
    else:
        raise ValueError("Format de fichier non supporté. Utilisez .txt ou .pdf")

# Fonction pour charger plusieurs fichiers
def load_multiple_files(file_paths):
    all_texts = []
    file_sources = []

    for file_path in file_paths:
        print(f"Chargement du fichier : {file_path}")
        try:
            text = load_file(file_path)
            all_texts.append(text)
            file_sources.append(os.path.basename(file_path))
            print(f"✅ Fichier chargé avec succès : {len(text)} caractères")
        except Exception as e:
            print(f"❌ Erreur lors du chargement de {file_path}: {e}")

    return all_texts, file_sources

# Fonction pour créer des embeddings
def get_text_embedding(input):
    response = client.embeddings.create(
        model="mistral-embed",
        inputs=input
    )
    return response.data[0].embedding

In [41]:
# AJOUTEZ VOS FICHIERS ICI
file_paths = [
    '/content/scan document001.pdf'
    '/content/POUR LES METIERS DE LA CAISSE DE SECUTITE SOCIALE.pdf',
    '/content/faq_metiers.pdf',
    # Ajoutez autant de fichiers que vous voulez
]

print("Fichiers à traiter :", file_paths)

Fichiers à traiter : ['/content/scan document001.pdf/content/POUR LES METIERS DE LA CAISSE DE SECUTITE SOCIALE.pdf', '/content/faq_metiers.pdf']


In [42]:
# Charger tous les fichiers
all_texts, file_sources = load_multiple_files(file_paths)

# Paramètres
chunk_size = 2048

# Créer les chunks avec leur source
all_chunks = []
chunk_sources = []

for i, text in enumerate(all_texts):
    file_chunks = [text[j:j + chunk_size] for j in range(0, len(text), chunk_size)]
    all_chunks.extend(file_chunks)
    chunk_sources.extend([file_sources[i]] * len(file_chunks))

print("Nombre total de blocs :", len(all_chunks))
print("Fichiers traités :", file_sources)

Chargement du fichier : /content/scan document001.pdf/content/POUR LES METIERS DE LA CAISSE DE SECUTITE SOCIALE.pdf
❌ Erreur lors du chargement de /content/scan document001.pdf/content/POUR LES METIERS DE LA CAISSE DE SECUTITE SOCIALE.pdf: [Errno 20] Not a directory: '/content/scan document001.pdf/content/POUR LES METIERS DE LA CAISSE DE SECUTITE SOCIALE.pdf'
Chargement du fichier : /content/faq_metiers.pdf
✅ Fichier chargé avec succès : 1639 caractères
Nombre total de blocs : 1
Fichiers traités : ['faq_metiers.pdf']


In [43]:
# Sauvegarder les chunks et leurs sources
with open('rag_data.pkl', 'wb') as f:
    pickle.dump({
        'chunks': all_chunks,
        'chunk_sources': chunk_sources,
        'file_sources': file_sources
    }, f)

# Sauvegarder aussi en format texte lisible
combined_text = "\n\n--- NOUVEAU FICHIER ---\n\n".join(all_texts)
with open('loaded_texts_combined.txt', 'w', encoding='utf-8') as f:
    f.write(combined_text)

print("✅ Données textuelles sauvegardées dans 'rag_data.pkl' et 'loaded_texts_combined.txt'")

✅ Données textuelles sauvegardées dans 'rag_data.pkl' et 'loaded_texts_combined.txt'


In [44]:
# =============================================================================
print("Création des embeddings...")
text_embeddings = np.array([get_text_embedding(chunk) for chunk in all_chunks])

# Créer et remplir l'index FAISS
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

print("✅ Base vectorielle créée avec succès !")
print(f"Dimensions des embeddings : {d}")
print(f"Nombre de vecteurs dans l'index : {index.ntotal}")

Création des embeddings...
✅ Base vectorielle créée avec succès !
Dimensions des embeddings : 1024
Nombre de vecteurs dans l'index : 1


In [45]:
# =============================================================================
# Sauvegarder l'index FAISS
faiss.write_index(index, "rag_faiss_index.index")

# Sauvegarder les embeddings aussi
np.save('text_embeddings.npy', text_embeddings)

print("✅ Index FAISS sauvegardé dans 'rag_faiss_index.index'")
print("✅ Embeddings sauvegardés dans 'text_embeddings.npy'")

✅ Index FAISS sauvegardé dans 'rag_faiss_index.index'
✅ Embeddings sauvegardés dans 'text_embeddings.npy'


In [46]:
def poser_question(question):
    question_embeddings = np.array([get_text_embedding(question)])
    D, I = index.search(question_embeddings, k=2)
    retrieved_chunks = []
    sources = []

    for i in I[0].tolist():
        retrieved_chunks.append(all_chunks[i])
        sources.append(chunk_sources[i])

    return retrieved_chunks, sources

def generer_reponse(question):
    retrieved_chunks, sources = poser_question(question)

    prompt = f"""
Context information is below.
---------------------
{retrieved_chunks[0]}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""

    response = client.chat.complete(
        model="mistral-large-latest",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content, sources

print("✅ Fonctions de recherche définies")

✅ Fonctions de recherche définies


In [53]:
# =============================================================================
question = "Quels sont les documents à fournir pour une demande d’immatriculation pour un employeur du régime domestique ?"
reponse, sources = generer_reponse(question)

print("QUESTION :", question)
print("RÉPONSE :")
print(reponse)


QUESTION : Quels sont les documents à fournir pour une demande d’immatriculation pour un employeur du régime domestique ?
RÉPONSE :
D'après le contexte fourni, les documents à fournir pour une demande d'immatriculation pour un employeur du régime domestique ne sont pas explicitement détaillés. Cependant, en se basant sur la liste générale des documents demandés pour les employeurs, voici ceux qui pourraient s'appliquer à un employeur de personnel domestique :

- **Copie de la Carte Nationale d’Identité (CNI) ou extrait de naissance** (pour l'employeur et les salariés).
- **Contrats de travail** (si applicable).
- **Déclaration de mouvement de travailleur visée par l’Inspection du travail** (si nécessaire).

Les autres documents mentionnés (comme les statuts, le NINEA, ou le registre de commerce) concernent davantage les entreprises ou structures formelles, et ne sont donc probablement pas requis pour un employeur domestique.

Pour une réponse précise, il serait conseillé de consulter d

In [ ]:
print("\n" + "="*50)
print("🎉 VOTRE RAG EST MAINTENANT PRÊT ET SAUVEGARDÉ !")
print("Vous pouvez poser d'autres questions :")
print("="*50)

def mode_interactif():
    while True:
        nouvelle_question = input("\nPosez votre question (ou 'quit' pour arrêter) : ")
        if nouvelle_question.lower() in ['quit', 'exit', 'stop']:
            break

        reponse, sources = generer_reponse(nouvelle_question)

        print("\nRÉPONSE :")
        print(reponse)
        print(f"Source(s) : {', '.join(set(sources))}")


🎉 VOTRE RAG EST MAINTENANT PRÊT ET SAUVEGARDÉ !
Vous pouvez poser d'autres questions :


In [ ]:
# =============================================================================
def recharger_rag():
    """
    Fonction pour recharger le RAG dans une nouvelle session
    """
    global all_chunks, chunk_sources, file_sources, index, text_embeddings

    # Recharger les données
    with open('rag_data.pkl', 'rb') as f:
        data = pickle.load(f)
        all_chunks = data['chunks']
        chunk_sources = data['chunk_sources']
        file_sources = data['file_sources']

    # Recharger l'index FAISS
    index = faiss.read_index("rag_faiss_index.index")

    # Recharger les embeddings
    text_embeddings = np.load('text_embeddings.npy')

    print("✅ RAG rechargé avec succès !")
    print(f"Chunks : {len(all_chunks)}")
    print(f"Index FAISS : {index.ntotal} vecteurs")
    print(f"Fichiers sources : {file_sources}")

    return True

print("✅ Fonction de rechargement définie")
print("\nPour recharger dans une nouvelle session, utilisez : recharger_rag()")

In [ ]:
# CELLULE 14 : Résumé et instructions
# =============================================================================
print("\n" + "="*60)
print("📋 RÉSUMÉ DE VOTRE RAG MULTI-FICHIERS")
print("="*60)
print(f"✅ Fichiers traités : {len(file_sources)}")
print(f"✅ Chunks créés : {len(all_chunks)}")
print(f"✅ Vecteurs dans l'index : {index.ntotal}")
print(f"✅ Fichiers sauvegardés :")
print("   - rag_data.pkl (chunks et métadonnées)")
print("   - rag_faiss_index.index (index vectoriel)")
print("   - text_embeddings.npy (embeddings)")
print("   - loaded_texts_combined.txt (texte combiné)")

print("\n📝 POUR LES PROCHAINES SESSIONS :")
print("1. Exécutez les cellules 1-4 (imports et fonctions)")
print("2. Exécutez la cellule 3 (authentification)")
print("3. Exécutez la cellule 13 puis appelez recharger_rag()")
print("4. Utilisez generer_reponse(votre_question)")

print("\n🔄 POUR AJOUTER DE NOUVEAUX FICHIERS :")
print("1. Modifiez la liste file_paths dans la cellule 5")
print("2. Réexécutez les cellules 5-12")